In [341]:
'''
Description: Notebook for cleaning and preprocessing the population data.
Version: 1.0
Autor: 
Date: 2021-08-09 19:02:46
LastEditors: 
LastEditTime: 2021-11-04 14:54:04
'''

'\nDescription: Notebook for cleaning and preprocessing the population data.\nVersion: 1.0\nAutor: Yaxiong Shao\nDate: 2021-08-09 19:02:46\nLastEditors: Yaxiong Shao\nLastEditTime: 2021-11-04 14:54:04\n'

In [1]:
import pandas as pd
import glob

In [2]:
pd.__version__ #make sure pandas version is greater than 1.2.0 to use cross merge

'1.3.4'

In [3]:
# read population data from csv files
csv_list=glob.glob("Data/DECENNIALSF12010.P12*.csv")
df_list=[]
for csv in csv_list:
    df_list.append(pd.read_csv(csv,skiprows = [1])) #skip second row which contains column names

df=pd.concat(df_list)
df.head()

/Users/yaxiongshao/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,GEO_ID,NAME,P012001,P012002,P012003,P012004,P012005,P012006,P012007,P012008,...,P012040,P012041,P012042,P012043,P012044,P012045,P012046,P012047,P012048,P012049
0,1000000US550019501001000,"Block 1000, Block Group 1, Census Tract 9501, ...",5,3,0,0,0,0,1,0,...,0,0,0,0,0,0,2,0,0,0
1,1000000US550019501001001,"Block 1001, Block Group 1, Census Tract 9501, ...",7,4,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1000000US550019501001002,"Block 1002, Block Group 1, Census Tract 9501, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000000US550019501001003,"Block 1003, Block Group 1, Census Tract 9501, ...",11,6,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4,1000000US550019501001004,"Block 1004, Block Group 1, Census Tract 9501, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# clean header names and drop unnecessary columns
df[['V','GEOID']] = df['GEO_ID'].str.split('S',expand=True)
df["GEO_ID"]=df["GEOID"] 
df["GEO_ID"] =df["GEO_ID"].astype("int64")
df.drop(["GEOID","NAME","V"],axis=1,inplace=True)

col=["GEOID", "Total", "M", "MUnder5", "M59", "M1014", "M1517", "M1819", "M20", "M21", "M2224", "M2529", "M3034", "M3539", "M4044", "M4549", "M5054", "M5559", "M6061", "M6264", "M6566", "M6769", "M7074", "M7579", "M8084", "M85over", "F", "FUnder5", "F59", "F1014", "F1517", "F1819", "F20", "F21", "F2224", "F2529", "F3034", "F3539", "F4044", "F4549", "F5054", "F5559", "F6061", "F6264", "F6566", "F6769", "F7074", "F7579", "F8084", "F85over"]
df.columns =col
df.head()

,GEOID,Total,M,MUnder5,M59,M1014,M1517,M1819,M20,M21,...,F5054,F5559,F6061,F6264,F6566,F6769,F7074,F7579,F8084,F85over
0,550019501001000,5,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,2,0,0,0
1,550019501001001,7,4,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,550019501001002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,550019501001003,11,6,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4,550019501001004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 2. Filter data only in the study area

In [5]:
#load the preselected census blocks for the study area
pop_in_study_area = pd.read_csv("Data/cook_blocks_2020_15mibuffer_popV6_166980.csv")
pop_in_study_area.head()

,BLOCKID10,POP10
0,170318253023034,24
1,170318255011022,0
2,170318253023026,11
3,170318253023045,12
4,170318253023018,5


In [6]:
#using inner join to merge the population data with the study area census blocks data to select data in our study area
pop = df.merge(pop_in_study_area,left_on="GEOID",right_on="BLOCKID10")
pop.head()

,GEOID,Total,M,MUnder5,M59,M1014,M1517,M1819,M20,M21,...,F6061,F6264,F6566,F6769,F7074,F7579,F8084,F85over,BLOCKID10,POP10
0,180890101001001,25,13,1,0,0,0,0,0,0,...,0,3,1,0,1,0,0,0,180890101001001,25
1,180890101001002,67,32,0,1,1,0,0,0,0,...,2,2,1,4,2,0,0,0,180890101001002,67
2,180890101002014,151,75,6,4,5,2,2,0,0,...,2,5,3,2,5,2,2,2,180890101002014,151
3,180890101002015,40,15,0,0,2,0,1,0,0,...,2,3,0,2,0,1,1,2,180890101002015,40
4,180890101002016,55,25,1,1,0,0,0,0,0,...,3,3,0,2,1,2,0,0,180890101002016,55


In [7]:
# re-group the data
pop["M014"] = pop["MUnder5"]+pop["M59"]+pop["M1014"]
pop["F014"] = pop["FUnder5"]+pop["F59"]+pop["F1014"]

pop["M1519"] = pop["M1517"]+pop["M1819"]
pop["F1519"] = pop["F1517"]+pop["F1819"]

pop["M2024"] = pop["M20"] + pop["M21"] +pop["M2224"]
pop["F2024"] = pop["F20"] + pop["F21"] +pop["F2224"]

pop["M6064"]=pop["M6061"] + pop["M6264"] 
pop["F6064"]=pop["F6061"] +pop["F6264"]

pop["M6569"]=pop["M6566"] + pop["M6769"] 
pop["F6569"]=pop["F6566"] +pop["F6769"]


drop_m = ["MUnder5","M59","M1014","M1517","M1819","M20","M21","M2224","M6566","M6769","M6061","M6264"]
drop_f = ["FUnder5","F59","F1014","F1517","F1819","F20","F21","F2224","F6566","F6769","F6061","F6264"]

pop.drop(drop_m,axis=1,inplace=True)
pop.drop(drop_f,axis=1,inplace=True)
pop.drop(["M","BLOCKID10","POP10","F"],axis=1,inplace=True)
pop.head()


,GEOID,Total,M2529,M3034,M3539,M4044,M4549,M5054,M5559,M7074,...,M014,F014,M1519,F1519,M2024,F2024,M6064,F6064,M6569,F6569
0,180890101001001,25,0,0,1,0,0,3,2,0,...,1,1,0,0,0,1,1,3,2,1
1,180890101001002,67,0,2,4,1,1,1,4,2,...,2,6,0,1,0,0,4,4,4,5
2,180890101002014,151,3,3,3,3,6,5,4,4,...,15,15,4,6,4,2,6,7,8,5
3,180890101002015,40,1,1,0,2,1,0,2,2,...,2,1,1,1,0,1,3,5,0,2
4,180890101002016,55,2,1,2,1,1,1,4,4,...,2,3,0,2,0,1,3,6,2,2


In [8]:
pop.columns

Index(['GEOID', 'Total', 'M2529', 'M3034', 'M3539', 'M4044', 'M4549', 'M5054',
       'M5559', 'M7074', 'M7579', 'M8084', 'M85over', 'F2529', 'F3034',
       'F3539', 'F4044', 'F4549', 'F5054', 'F5559', 'F7074', 'F7579', 'F8084',
       'F85over', 'M014', 'F014', 'M1519', 'F1519', 'M2024', 'F2024', 'M6064',
       'F6064', 'M6569', 'F6569'],
      dtype='object')

In [9]:
# re-order the columns
new_order=['GEOID', 'Total', 'F014', 'F1519', 'F2024', 'F2529', 'F3034', 'F3539', 'F4044', 'F4549', 'F5054', 'F5559', 'F6064', 'F6569','F7074', 'F7579', 'F8084', 'F85over', 'M014', 'M1519', 'M2024', 'M2529', 'M3034', 'M3539', 'M4044', 'M4549', 'M5054', 'M5559', 'M6064', 'M6569', 'M7074', 'M7579', 'M8084', 'M85over']
pop=pop[new_order]
pop.head()

,GEOID,Total,F014,F1519,F2024,F2529,F3034,F3539,F4044,F4549,...,M4044,M4549,M5054,M5559,M6064,M6569,M7074,M7579,M8084,M85over
0,180890101001001,25,1,0,1,0,0,1,0,0,...,0,0,3,2,1,2,0,1,1,1
1,180890101001002,67,6,1,0,1,0,1,4,2,...,1,1,1,4,4,4,2,7,0,0
2,180890101002014,151,15,6,2,2,4,6,3,3,...,3,6,5,4,6,8,4,4,2,1
3,180890101002015,40,1,1,1,1,0,0,0,3,...,2,1,0,2,3,0,2,0,0,0
4,180890101002016,55,3,2,1,1,1,1,0,3,...,1,1,1,4,3,2,4,2,0,0


# 3. Prepare the multiples

In [10]:
#insurance premium data
PremiumMultiple_df=pd.read_csv("Data/AgePremiumMultiple.csv")
PremiumMultiple_df.head(20)

,Age,PremiumMultiple
0,0-14,0.770
1,15-19,0.886
2,20-24,0.994
3,24-29,1.058
4,30-34,1.178
5,35-39,1.240
6,40-44,1.334
7,45-49,1.570
8,50-54,1.958
9,55-59,2.430


In [11]:
# gender demand data
age_gender_df=pd.read_csv("Data/PercapitaSpendingbyAgeGenderV4.csv")
age_gender_df.head(20)

,Age,MaleMultiple,FemaleMultiple
0,0-14,1.09,1.01
1,15-19,1.09,1.01
2,20-24,1.00,1.66
3,24-29,1.00,1.66
4,30-34,1.00,1.66
5,35-39,1.00,1.66
6,40-44,1.00,1.66
7,45-49,2.77,2.99
8,50-54,2.77,2.99
9,55-59,2.77,2.99


In [12]:
#merge multiples into one dataframe
premium_age_gender_df= PremiumMultiple_df.merge(age_gender_df,on="Age")
premium_age_gender_df.head(20)

,Age,PremiumMultiple,MaleMultiple,FemaleMultiple
0,0-14,0.770,1.09,1.01
1,15-19,0.886,1.09,1.01
2,20-24,0.994,1.00,1.66
3,24-29,1.058,1.00,1.66
4,30-34,1.178,1.00,1.66
5,35-39,1.240,1.00,1.66
6,40-44,1.334,1.00,1.66
7,45-49,1.570,2.77,2.99
8,50-54,1.958,2.77,2.99
9,55-59,2.430,2.77,2.99


In [13]:
pop.columns

Index(['GEOID', 'Total', 'F014', 'F1519', 'F2024', 'F2529', 'F3034', 'F3539',
       'F4044', 'F4549', 'F5054', 'F5559', 'F6064', 'F6569', 'F7074', 'F7579',
       'F8084', 'F85over', 'M014', 'M1519', 'M2024', 'M2529', 'M3034', 'M3539',
       'M4044', 'M4549', 'M5054', 'M5559', 'M6064', 'M6569', 'M7074', 'M7579',
       'M8084', 'M85over'],
      dtype='object')

In [14]:
#append Multiples to the population dataframe 
for index,row in premium_age_gender_df.iterrows():
    pop[row["Age"]+"_PM"]= row["PremiumMultiple"]
    pop[row["Age"]+"_MM"]= row["MaleMultiple"]
    pop[row["Age"]+"_FM"]= row["FemaleMultiple"]

pop.head()

,GEOID,Total,F014,F1519,F2024,F2529,F3034,F3539,F4044,F4549,...,70-74_FM,75-79_PM,75-79_MM,75-79_FM,80-84_PM,80-84_MM,80-84_FM,85 and over_PM,85 and over_MM,85 and over_FM
0,180890101001001,25,1,0,1,0,0,1,0,0,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
1,180890101001002,67,6,1,0,1,0,1,4,2,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
2,180890101002014,151,15,6,2,2,4,6,3,3,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
3,180890101002015,40,1,1,1,1,0,0,0,3,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
4,180890101002016,55,3,2,1,1,1,1,0,3,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67


In [15]:
pop.columns

Index(['GEOID', 'Total', 'F014', 'F1519', 'F2024', 'F2529', 'F3034', 'F3539',
       'F4044', 'F4549', 'F5054', 'F5559', 'F6064', 'F6569', 'F7074', 'F7579',
       'F8084', 'F85over', 'M014', 'M1519', 'M2024', 'M2529', 'M3034', 'M3539',
       'M4044', 'M4549', 'M5054', 'M5559', 'M6064', 'M6569', 'M7074', 'M7579',
       'M8084', 'M85over', '0-14_PM', '0-14_MM', '0-14_FM', '15-19_PM',
       '15-19_MM', '15-19_FM', '20-24_PM', '20-24_MM', '20-24_FM', '24-29_PM',
       '24-29_MM', '24-29_FM', '30-34_PM', '30-34_MM', '30-34_FM', '35-39_PM',
       '35-39_MM', '35-39_FM', '40-44_PM', '40-44_MM', '40-44_FM', '45-49_PM',
       '45-49_MM', '45-49_FM', '50-54_PM', '50-54_MM', '50-54_FM', '55-59_PM',
       '55-59_MM', '55-59_FM', '60-64_PM', '60-64_MM', '60-64_FM', '65-69_PM',
       '65-69_MM', '65-69_FM', '70-74_PM', '70-74_MM', '70-74_FM', '75-79_PM',
       '75-79_MM', '75-79_FM', '80-84_PM', '80-84_MM', '80-84_FM',
       '85 and over_PM', '85 and over_MM', '85 and over_FM'],
      dty

# 4. Compute the result by multiply the multiple to pop data

In [16]:
res_df = pop.copy() #create a copy of the population dataframe

In [17]:
# multiply the population by the multiples to get the fixed population
res_df['F014']=pop['F014']*pop['0-14_FM']*pop['0-14_PM']
res_df['F1519']=pop['F1519']*pop['15-19_FM']*pop['15-19_PM']
res_df['F2024']=pop['F2024']*pop['20-24_FM']*pop['20-24_PM']
res_df['F2529']=pop['F2529']*pop['24-29_FM']*pop['24-29_PM']
res_df['F3034']=pop['F3034']*pop['30-34_FM']*pop['30-34_PM']
res_df['F3539']=pop['F3539']*pop['35-39_FM']*pop['35-39_PM']
res_df['F4044']=pop['F4044']*pop['40-44_FM']*pop['40-44_PM']
res_df['F4549']=pop['F4549']*pop['45-49_FM']*pop['45-49_PM']
res_df['F5054']=pop['F5054']*pop['50-54_FM']*pop['50-54_PM']
res_df['F5559']=pop['F5559']*pop['55-59_FM']*pop['55-59_PM']
res_df['F6064']=pop['F6064']*pop['60-64_FM']*pop['60-64_PM']
res_df['F6569']=pop['F6569']*pop['65-69_FM']*pop['65-69_PM']
res_df['F7074']=pop['F7074']*pop['70-74_FM']*pop['70-74_PM']
res_df['F7579']=pop['F7579']*pop['75-79_FM']*pop['75-79_PM']
res_df['F8084']=pop['F8084']*pop['80-84_FM']*pop['80-84_PM']
res_df['F85over']=pop['F85over']*pop['85 and over_FM']*pop['85 and over_PM']
res_df['M014']=pop['M014']*pop['0-14_MM']*pop['0-14_PM']
res_df['M1519']=pop['M1519']*pop['15-19_MM']*pop['15-19_PM']
res_df['M2024']=pop['M2024']*pop['20-24_MM']*pop['20-24_PM']
res_df['M2529']=pop['M2529']*pop['24-29_MM']*pop['24-29_PM']
res_df['M3034']=pop['M3034']*pop['30-34_MM']*pop['30-34_PM']
res_df['M3539']=pop['M3539']*pop['35-39_MM']*pop['35-39_PM']
res_df['M4044']=pop['M4044']*pop['40-44_MM']*pop['40-44_PM']
res_df['M4549']=pop['M4549']*pop['45-49_MM']*pop['45-49_PM']
res_df['M5054']=pop['M5054']*pop['50-54_MM']*pop['50-54_PM']
res_df['M5559']=pop['M5559']*pop['55-59_MM']*pop['55-59_PM']
res_df['M6064']=pop['M6064']*pop['60-64_MM']*pop['60-64_PM']
res_df['M6569']=pop['M6569']*pop['65-69_MM']*pop['65-69_PM']
res_df['M7074']=pop['M7074']*pop['70-74_MM']*pop['70-74_PM']
res_df['M7579']=pop['M7579']*pop['75-79_MM']*pop['75-79_PM']
res_df['M8084']=pop['M8084']*pop['80-84_MM']*pop['80-84_PM']
res_df['M85over']=pop['M85over']*pop['85 and over_MM']*pop['85 and over_PM']

res_df.head()

,GEOID,Total,F014,F1519,F2024,F2529,F3034,F3539,F4044,F4549,...,70-74_FM,75-79_PM,75-79_MM,75-79_FM,80-84_PM,80-84_MM,80-84_FM,85 and over_PM,85 and over_MM,85 and over_FM
0,180890101001001,25,0.7777,0.00000,1.65004,0.00000,0.00000,2.0584,0.00000,0.0000,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
1,180890101001002,67,4.6662,0.89486,0.00000,1.75628,0.00000,2.0584,8.85776,9.3886,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
2,180890101002014,151,11.6655,5.36916,3.30008,3.51256,7.82192,12.3504,6.64332,14.0829,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
3,180890101002015,40,0.7777,0.89486,1.65004,1.75628,0.00000,0.0000,0.00000,14.0829,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67
4,180890101002016,55,2.3331,1.78972,1.65004,1.75628,1.95548,2.0584,0.00000,14.0829,...,5.19,3.0,5.23,5.19,3.0,5.23,5.19,3.0,9.45,10.67


In [18]:
res_df.columns

Index(['GEOID', 'Total', 'F014', 'F1519', 'F2024', 'F2529', 'F3034', 'F3539',
       'F4044', 'F4549', 'F5054', 'F5559', 'F6064', 'F6569', 'F7074', 'F7579',
       'F8084', 'F85over', 'M014', 'M1519', 'M2024', 'M2529', 'M3034', 'M3539',
       'M4044', 'M4549', 'M5054', 'M5559', 'M6064', 'M6569', 'M7074', 'M7579',
       'M8084', 'M85over', '0-14_PM', '0-14_MM', '0-14_FM', '15-19_PM',
       '15-19_MM', '15-19_FM', '20-24_PM', '20-24_MM', '20-24_FM', '24-29_PM',
       '24-29_MM', '24-29_FM', '30-34_PM', '30-34_MM', '30-34_FM', '35-39_PM',
       '35-39_MM', '35-39_FM', '40-44_PM', '40-44_MM', '40-44_FM', '45-49_PM',
       '45-49_MM', '45-49_FM', '50-54_PM', '50-54_MM', '50-54_FM', '55-59_PM',
       '55-59_MM', '55-59_FM', '60-64_PM', '60-64_MM', '60-64_FM', '65-69_PM',
       '65-69_MM', '65-69_FM', '70-74_PM', '70-74_MM', '70-74_FM', '75-79_PM',
       '75-79_MM', '75-79_FM', '80-84_PM', '80-84_MM', '80-84_FM',
       '85 and over_PM', '85 and over_MM', '85 and over_FM'],
      dty

In [19]:
#drop the columns after columns 34 which contain the multiples
drop_index=[x for x in range(34,len(res_df.columns))] 
res_df.drop(res_df.columns[drop_index],axis=1,inplace=True)
res_df.head()

,GEOID,Total,F014,F1519,F2024,F2529,F3034,F3539,F4044,F4549,...,M4044,M4549,M5054,M5559,M6064,M6569,M7074,M7579,M8084,M85over
0,180890101001001,25,0.7777,0.00000,1.65004,0.00000,0.00000,2.0584,0.00000,0.0000,...,0.000,0.0000,16.27098,13.4622,7.9499,31.38,0.00,15.69,15.69,28.35
1,180890101001002,67,4.6662,0.89486,0.00000,1.75628,0.00000,2.0584,8.85776,9.3886,...,1.334,4.3489,5.42366,26.9244,31.7996,62.76,31.38,109.83,0.00,0.00
2,180890101002014,151,11.6655,5.36916,3.30008,3.51256,7.82192,12.3504,6.64332,14.0829,...,4.002,26.0934,27.11830,26.9244,47.6994,125.52,62.76,62.76,31.38,28.35
3,180890101002015,40,0.7777,0.89486,1.65004,1.75628,0.00000,0.0000,0.00000,14.0829,...,2.668,4.3489,0.00000,13.4622,23.8497,0.00,31.38,0.00,0.00,0.00
4,180890101002016,55,2.3331,1.78972,1.65004,1.75628,1.95548,2.0584,0.00000,14.0829,...,1.334,4.3489,5.42366,26.9244,23.8497,31.38,62.76,31.38,0.00,0.00


In [20]:
#compute demand score by summing the columns for each row
sum_cols_index=[x for x in range(2,34)]
#res_df['demand_score3']=res_df.iloc[:,sum_cols_index].sum(axis=1)
res_df['demand_score']=res_df[res_df.columns[sum_cols_index]].sum(axis=1)
res_df.drop(res_df.columns[sum_cols_index],axis=1,inplace=True)
res_df.head()

,GEOID,Total,demand_score
0,180890101001001,25,218.48266
1,180890101001002,67,514.88992
2,180890101002014,151,961.82218
3,180890101002015,40,318.40604
4,180890101002016,55,397.85432


In [21]:
#save results to csv for number of population used for the comparison analysis
res_df.columns=['geoid','pop','ds']
res_df.to_csv("Results/cook_blocks_2020_15mibuffer_pop_demandscore_V9_166980.csv",index=False)

In [ ]:
res_df.drop(['pop'],axis=1,inplace=True)

# 5. Compute the demand score by insurance


In [24]:
marketShare_df = pd.read_csv("Data/us_insuranceMarketshare_plan_matched_cleaned.csv")
marketShare_df["groupcode"]=marketShare_df["groupcode"].astype(int)
marketShare_df.head()

,groupcode,marketshare
0,-9999,0.2443
1,707,0.1413
2,601,0.0833
3,671,0.0615
4,1295,0.0600


In [25]:
marketShare_df.shape,res_df.shape,marketShare_df.shape[0]*res_df.shape[0]

((61, 2), (166980, 2), 10185780)

In [26]:
res_df2=res_df.merge(marketShare_df,how="cross")
res_df2.head()

,geoid,ds,groupcode,marketshare
0,180890101001001,218.48266,-9999,0.2443
1,180890101001001,218.48266,707,0.1413
2,180890101001001,218.48266,601,0.0833
3,180890101001001,218.48266,671,0.0615
4,180890101001001,218.48266,1295,0.0600


In [27]:
assert res_df2.shape[0] == ( marketShare_df.shape[0]*res_df.shape[0])

In [28]:
res_df2['ds']=res_df2['ds']*res_df2['marketshare']
res_df2.head()
res_df2.drop(['marketshare'],axis=1,inplace=True)
res_df2.to_csv("Results/cook_blocks_2020_15mibuffer_pop_demandscore_V9_166980_byInsurances.csv",index=False)